In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

In [3]:
data = gpd.read_file('OSM/armenia-latest-free.shp/gis_osm_landuse_a_free_1.shp')
for file in tqdm(['belarus', 'kazakhstan', 'central-fed-district',
             'crimean-fed-district', 'far-eastern-fed-district', 'north-caucasus-fed-district',
             'northwestern-fed-district', 'siberian-fed-district', 'south-fed-district',
             'ural-fed-district', 'volga-fed-district']):

    data1 = gpd.read_file('OSM/%s-latest-free.shp/gis_osm_landuse_a_free_1.shp' %file)
    data = pd.concat([data, data1], ignore_index=True)
    
    
    
data = gpd.GeoDataFrame(data, geometry='geometry')

100%|██████████| 11/11 [05:52<00:00, 32.05s/it]


In [14]:
train = pd.read_csv('train.csv')
train['type'] = 'train'
test = pd.read_csv('test.csv')
test['type'] = 'test'
points = pd.concat([train, test], ignore_index = True)
points

,id,sales_per_month,region,locality,x,y,people_iso_3,people_iso_5,people_iso_10,people_iso_15,people_iso_30,type
0,18398,2538.0,Челябинская область,Озерск,60.692229,55.761933,971,2093,10439,23027,73676,train
1,14532,1645.0,Москва,Москва,37.678559,55.702931,840,2329,15949,35836,67185,train
2,23801,3176.0,Липецкая область,Липецк,39.524007,52.589588,690,4012,22279,47864,163666,train
3,9791,4759.0,Краснодарский край,посёлок городского типа Сириус,39.957405,43.414255,49,515,2699,3710,9449,train
4,3004,2051.0,Тюменская область,Сургут,73.450188,61.237837,1416,5619,23135,50801,110709,train
...,...,...,...,...,...,...,...,...,...,...,...,...
26012,18085,NaN,Московская область,Воскресенск,38.693206,55.310078,0,0,0,0,8053,test
26013,23428,NaN,Ереван,Ереван,44.525006,40.131853,838,1136,9215,20182,47220,test
26014,11862,NaN,Лорийская область,Ванадзор,44.483527,40.811495,1234,3135,15032,28339,48335,test
26015,10382,NaN,Ереван,Ереван,44.525365,40.207058,1942,5353,23017,51741,147016,test


In [15]:
geometry = [Point(lon, lat) for lon, lat in zip(points['x'], points['y'])]

points = gpd.GeoDataFrame(points, geometry=geometry) #crs = EPSG:4326

In [16]:
joined_gdf = gpd.sjoin(points, data[['fclass', 'geometry']], how='left', op='within')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-16-aa5ad17280de>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  joined_gdf = gpd.sjoin(points, data[['fclass', 'geometry']], how='left', op='within')


In [17]:
joined_gdf

,id,sales_per_month,region,locality,x,y,people_iso_3,people_iso_5,people_iso_10,people_iso_15,people_iso_30,type,geometry,index_right,fclass
0,18398,2538.0,Челябинская область,Озерск,60.692229,55.761933,971,2093,10439,23027,73676,train,POINT (60.69223 55.76193),NaN,NaN
1,14532,1645.0,Москва,Москва,37.678559,55.702931,840,2329,15949,35836,67185,train,POINT (37.67856 55.70293),598516.0,residential
2,23801,3176.0,Липецкая область,Липецк,39.524007,52.589588,690,4012,22279,47864,163666,train,POINT (39.52401 52.58959),777617.0,residential
3,9791,4759.0,Краснодарский край,посёлок городского типа Сириус,39.957405,43.414255,49,515,2699,3710,9449,train,POINT (39.95741 43.41425),3681669.0,residential
4,3004,2051.0,Тюменская область,Сургут,73.450188,61.237837,1416,5619,23135,50801,110709,train,POINT (73.45019 61.23784),4153109.0,residential
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26013,23428,NaN,Ереван,Ереван,44.525006,40.131853,838,1136,9215,20182,47220,test,POINT (44.52501 40.13185),23380.0,residential
26014,11862,NaN,Лорийская область,Ванадзор,44.483527,40.811495,1234,3135,15032,28339,48335,test,POINT (44.48353 40.81150),15925.0,residential
26015,10382,NaN,Ереван,Ереван,44.525365,40.207058,1942,5353,23017,51741,147016,test,POINT (44.52537 40.20706),5104.0,residential
26015,10382,NaN,Ереван,Ереван,44.525365,40.207058,1942,5353,23017,51741,147016,test,POINT (44.52537 40.20706),21148.0,residential


In [18]:
joined_gdf.drop_duplicates(subset= ['id'], inplace = True)


In [19]:
new_train = joined_gdf[joined_gdf['type'] == 'train']
new_test = joined_gdf[joined_gdf['type'] == 'test']

new_train.to_csv('train_landuse.csv', index = False)
new_test.to_csv('test_landuse.csv', index = False)